# Assignment 1 - Power Plant Prediction
Kyle Ah Von #57862609

## Summary
Predict the Electrical Energy Output (EP) using: Multiple Regression and SVM Regression

## Dataset Details

- **Total Data Points:** 9,568
- **Time Period:** 2006-2011
- **Frequency:** Hourly averages

## Features

The dataset includes the following features:

1. **Temperature (T)**
   - **Description:** The temperature of the ambient environment.
   - **Range:** 1.81°C to 37.11°C

2. **Ambient Pressure (AP)**
   - **Description:** The ambient pressure at the location of the power plant.
   - **Range:** 992.89 to 1033.30 millibar

3. **Relative Humidity (RH)**
   - **Description:** The relative humidity of the ambient environment.
   - **Range:** 25.56% to 100.16%

4. **Exhaust Vacuum (V)**
   - **Description:** The vacuum pressure in the exhaust system.
   - **Range:** 25.36 to 81.56 cm Hg

5. **Net Hourly Electrical Energy Output (EP)**
   - **Description:** The net electrical energy output of the power plant per hour.
   - **Range:** 420.26 to 495.76 MW



# Importing Libraries

In [41]:
# Data Manipulation
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations

# Data Visualization
import matplotlib.pyplot as plt  # For creating static, interactive, and animated visualizations
import seaborn as sns            # For statistical data visualization

# Machine Learning
from sklearn.model_selection import train_test_split  # For splitting the dataset into training and testing sets
from sklearn.preprocessing import StandardScaler, MinMaxScaler , PolynomialFeatures # For feature scaling & polynomial regression
from sklearn.linear_model import LinearRegression  # For linear regression
from sklearn.svm import SVR  # For Support Vector Regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # For model evaluation metrics

## Import DataSet 

In [42]:
dataset = pd.read_csv('Power Plant Data.csv')

### Visualize dataset in table

In [43]:
# Set the float format to 2 decimal - data does not have more than 2 significant figures after decimal point
pd.options.display.float_format = '{:.2f}'.format

# Display the first few rows as sample
dataset.head()

,Ambient Temperature (C),Exhaust Vacuum (cm Hg),Ambient Pressure (milibar),Relative Humidity (%),Hourly Electrical Energy output (MW)
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


All the columns have numerical data. All the columns contain useful information for the model. 

## Review of Data
Primarily to check if we have missing data

In [44]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Ambient Temperature (C)               9568 non-null   float64
 1   Exhaust Vacuum (cm Hg)                9568 non-null   float64
 2   Ambient Pressure (milibar)            9568 non-null   float64
 3   Relative Humidity (%)                 9568 non-null   float64
 4   Hourly Electrical Energy output (MW)  9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


Conveniently, there is no missing data in any rows, woohoo!
Also, there is no categorical data, there is no need to encode the data.

## Separate Inputs and Outputs

In [45]:
Y = dataset.iloc[:, 4] # sets last column as output
X = dataset.iloc[:, :4] # sets the first 4 columns as input


## Split data into Training and Testing 

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

#Visualize the training input data
X_train.head()


,Ambient Temperature (C),Exhaust Vacuum (cm Hg),Ambient Pressure (milibar),Relative Humidity (%)
9061,6.61,38.91,1015.77,92.31
6937,22.72,65.61,1014.64,70.53
5631,10.06,39.61,1018.22,70.22
6218,27.53,67.83,1009.40,53.73
1362,23.89,48.41,1010.48,62.31


## Scale data - Standardization
Since each input feature has a difference range of values, scaling the data would reduce the bias on features with bigger absolute values. 

In [47]:
sc_x = StandardScaler()

#Fit and transform
X_train_scaled = sc_x.fit_transform(X_train)
X_test_scaled = sc_x.transform(X_test)

#View Scaled data
print(X_train_scaled)

[[-1.75440309 -1.21216858  0.42112637  1.3027435 ]
 [ 0.41388954  0.88854693  0.23226259 -0.1942825 ]
 [-1.29005737 -1.15709364  0.83060976 -0.21559003]
 ...
 [-0.1419794  -1.16260113 -0.78392477 -0.32625174]
 [ 0.44619185 -0.73222983 -0.17387808 -1.54971652]
 [ 0.11509316 -0.36401453 -0.61010325 -0.17503699]]


## Regressions

### Simple Linear Regression Model 
In the case of multiple inputs, the model assigns a separate coefficient to each of the input features which are Ambient Temperature, Exhaust Vaccum, Ambient Pressure and Relative Humidity. These coefficients represent the weight or importance of each feature in predicting the output. 

The prediction is made by taking a linear combination of these features. The model then uses an optimization algorithm to find the best values for these coefficients that minimize the prediction error across the training data. 

In [48]:
# Apply linear regression
LinearRegModel = LinearRegression()
LinearRegModel.fit(X_train_scaled, y_train)

# Predict on test data
y_simple_pred_train = LinearRegModel.predict(X_train_scaled)
y_simple_pred_test = LinearRegModel.predict(X_test_scaled)


### Polynomial Regression Model

In this section, we will up the complexity of the model by fitting a polynomial function. The code will iteration over a few polynomial degrees and spit out the results on a visualization. 

In [49]:
# Apply polynomial Regression

results = []

for degree in range(1, 8):
    #create and fit the polynomial regression model
    poly_reg = PolynomialFeatures(degree)
    X_poly = poly_reg.fit_transform(X_train_scaled)
    poly_model = LinearRegression()
    poly_model.fit(X_poly, y_train)

    # Make Predictions
    y_poly_pred_train = poly_model.predict(X_train_scaled)
    y_poly_pred_test = poly_model.predict(X_test_scaled)

    # Calculate metrics (r2_score)
    r2_train = r2_score(y_train, y_poly_pred_train)
    r2_test = r2_score(y_test, y_poly_pred_test)

    results.append({
        'Degree': degree,
        'R2 (Train)': r2_train, 
        'R2 (Test)': r2_test
    })

    #Convert results to a DataFrame
    results_df = pd.Dataframe(results)

    # Find the best model based on test R2 score
    best_model = results_df.loc[results_df['R2 (Test)'].idxmax()]
    

plt.figure(figsize=(12, 6))
plt.plot(results['Degree'], results['R2 (Train)'], label='R2 (Train)')
plt.plot(results['Degree'], results['R2 (Test)'], label='R2 (Test)')
plt.xlabel('Polynomial Degree')
plt.ylabel('R2 Score')
plt.title('R2 Score vs Polynomial Degree')
plt.legend()
plt.show()


ValueError: X has 4 features, but LinearRegression is expecting 5 features as input.

### Test model

In [37]:
Linearmse = mean_squared_error(y_test, y_pred)
LinearR2 = r2_score(y_test, y_pred)
